In [2]:
%gui qt

In [ ]:

o

In [1]:
import vispy
# use jupyter rfb 
print(vispy.sys_info())

RFBOutputContext()

Platform: Linux-5.13.0-28-generic-x86_64-with-glibc2.35
Python:   3.11.9 | packaged by conda-forge | (main, Apr 19 2024, 18:36:13) [GCC 12.3.0]
NumPy:    1.26.4
Backend:  jupyter_rfb
pyqt4:    None
pyqt5:    ('PyQt5', '5.15.9', '5.15.8')
pyqt6:    None
pyside:   None
pyside2:  None
pyside6:  None
pyglet:   None
glfw:     None
sdl2:     None
wx:       None
egl:      EGL 1.5 NVIDIA: OpenGL_ES OpenGL
osmesa:   None
tkinter:  None
jupyter_rfb: jupyter_rfb
_test:    None

GL version:  '4.6 (Compatibility Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2'
MAX_TEXTURE_SIZE: 16384
Extensions: 'GL_ARB_multisample GL_EXT_abgr GL_EXT_bgra GL_EXT_blend_color GL_EXT_blend_minmax GL_EXT_blend_subtract GL_EXT_copy_texture GL_EXT_subtexture GL_EXT_texture_object GL_EXT_vertex_array GL_EXT_compiled_vertex_array GL_EXT_texture GL_EXT_texture3D GL_IBM_rasterpos_clip GL_ARB_point_parameters GL_EXT_draw_range_elements GL_EXT_packed_pixels GL_EXT_point_parameters GL_EXT_rescale_normal GL_EXT_separate_specular_color G

In [ ]:
import sys
from vispy import app, gloo


In [6]:
# import sys
# from vispy import app, gloo

# class MyCanvas(app.Canvas):
#     def on_draw(self, event):
#         gloo.set_clear_color((0.2, 0.4, 0.6, 1.0))
#         gloo.clear()


# canvas = MyCanvas(keys='interactive')
# canvas.show()

# if __name__ == '__main__' and sys.flags.interactive == 0:
#     app.run()

RFBOutputContext()

In [12]:
import sys
from vispy import app, scene, visuals,use
from vispy.util.filter import gaussian_filter
import numpy as np
import vispy.io as io
# if offscreen 
# use egl or osmesa
try:
    use('egl')
except:
    pass
# canvas = scene.SceneCanvas(keys='interactive', title='Isocurve(s) overlayed '
#                            'over Random Image Example')

class IsoCanvas(scene.SceneCanvas):
    def __init__(self):
        scene.SceneCanvas.__init__(self,keys='interactive', title='Isocurve(s) overlayed '
                           'over Random Image Example'
                                  )
        
        self.show()
        # self.size =  800, 1600
        self.unfreeze()
        # Set up a viewbox to display the image with interactive pan/zoom
        self.view = self.central_widget.add_view()
        # Create the image
        img_data = np.empty((200, 100, 3), dtype=np.ubyte)
        noise = np.random.normal(size=(200, 100), loc=50, scale=150)
        noise = gaussian_filter(noise, (4, 4, 0))
        img_data[:] = noise[..., np.newaxis]
        image = scene.visuals.Image(img_data, parent=self.view.scene)
        # move image behind curves
        image.transform = visuals.transforms.STTransform(translate=(0, 0, 0.5))

# level and color setup
        levels = [40, 50, 60]
        color_lev = [(1, 0, 0, 1),
             (1, 0.5, 0, 1),
             (1, 1, 0, 1)]

# Create isocurve, make a child of the image to ensure the two are always
# aligned.
        curve = scene.visuals.Isocurve(noise, levels=levels, color_lev=color_lev,
                               parent=self.view.scene)

# Set 2D camera
        self.view.camera = scene.PanZoomCamera(aspect=1)
# the camera will scale to the contents in the scene
        self.view.camera.set_range()

canvas = IsoCanvas()
img=canvas.render()
io.write_png("res/iso_.png",img)


In [ ]:
# https://vispy.org/gallery/scene/instanced_mesh.html#sphx-glr-gallery-scene-instanced-mesh-py

In [6]:
help(canvas.show)

Help on method show in module vispy.app.canvas:

show(visible=True, run=False) method of __main__.IsoCanvas instance
    Show or hide the canvas
    
    Parameters
    ----------
    visible : bool
        Make the canvas visible.
    run : bool
        Run the backend event loop.



In [ ]:
# draw stars


In [9]:
from vispy import app, gloo
from vispy.gloo import Program
import vispy.plot as vp
import vispy.io as io

vertex = """
    attribute vec4 color;
    attribute vec2 position;
    varying vec4 v_color;
    void main()
    {
        gl_Position = vec4(position, 0.0, 1.0);
        v_color = color;
    } """

fragment = """
    varying vec4 v_color;
    void main()
    {
        gl_FragColor = v_color;
    } """


class Canvas(app.Canvas):
    def __init__(self):
        super().__init__(size=(512, 512), title='Colored quad',
                         keys='interactive')

        # Build program
        self.program = Program(vertex, fragment, count=4)

        # Set uniforms and attributes
        self.program['color'] = [(1, 0, 0, 1), (0, 1, 0, 1),
                                 (0, 0, 1, 1), (1, 1, 0, 1)]
        self.program['position'] = [(-1, -1), (-1, +1),
                                    (+1, -1), (+1, +1)]

        gloo.set_viewport(0, 0, *self.physical_size)

        self.show()

    def on_draw(self, event):
        gloo.clear()
        self.program.draw('triangle_strip')

    def on_resize(self, event):
        gloo.set_viewport(0, 0, *event.physical_size)

if __name__ == '__main__':
    # c = Canvas()
    # app.run()
    c = Canvas()
    img = c.render()
    

RFBOutputContext()

In [4]:
import numpy as np
from vispy import scene
# from vispy.scene import visuals
from vispy import visuals
from vispy.app import Application

# Specify the backend explicitly
app = Application(backend_name='pyqt5')  # Options include 'pyqt5', 'wx', 'tk', 'osmesa', etc.
# Create a canvas and plot
canvas = scene.SceneCanvas(keys='interactive', show=True)
view = canvas.central_widget.add_view()

# Data
n = 100
x = np.random.rand(n) * 10
y = np.random.rand(n) * 10
size = np.random.rand(n) * 100  # Base size for ellipses

# Create ellipses
ellipses = visuals.ellipse.EllipseCollection()
for i in range(n):
    width = y[i] * 0.2
    ellipses.append([x[i], y[i]], width, 1)  # width varies by y, height is constant

# Add to canvas
view.add(ellipses)
view.camera = scene.cameras.PanZoomCamera(rect=(0, 0, 10, 10))

canvas.app.run()


RFBOutputContext()

AttributeError: module 'vispy.visuals.ellipse' has no attribute 'EllipseCollection'

In [4]:
help(canvas.on_draw)

Help on method on_draw in module vispy.scene.canvas:

on_draw(event) method of vispy.scene.canvas.SceneCanvas instance
    Draw handler
    
    Parameters
    ----------
    event : instance of Event
        The draw event.

